In [ ]:
def c(s):
    w = 0
    a = 0
    t = 0
    e = 0
    r = 0
    l = 0
    o = 0
    x = True
    o_2 = 0


    for i in range(0, len(s)):
        if s[i] == 'w':
            w = w + 1
        elif s[i] == 'a':
            a = a + w
        elif s[i] == 't':
            t = t + a
        elif s[i] == 'e':
            e = e + t
        elif s[i] == 'r':
            r = r + e
        elif s[i] == 'l':
            l = l + r
        elif s[i] == 'o':
            if x:
                x = False
            else:
                o_2 = o_2 + o
            o = o + l
    return o_2

In [ ]:
# basic sanity check

assert c('waterlo') == 0
assert c('waterloo') == 1
assert c('aterloowaterloo') == 1
assert c('wwaterloo') == 2
assert c('waaterrloo') == 4
assert c('waterlooo') == 3

In [ ]:
import random
import string
import numpy as np

from statistics import mean
from math import comb


def avg(n):
  return comb(n,8) / 26 ** 8

def ta(n,t):
  l = []
  for _ in range(n):
    l.append(c(np.array([random.choice(string.ascii_lowercase) for _ in range(t)])))
  return mean(l)

print(f'{ta(100,132680):.25f}')
print(f'{avg(132680):.25f}')
  

11461131697341213244915712.0000000000000000000000000
11403672671350719854411776.0000000000000000000000000


So random strings do seem to contain WATERLOO as often as we expected. 

Now let's try our hands on Hamlet

In [ ]:
import urllib
import re

h = urllib.request.urlopen("https://gist.githubusercontent.com/provpup/2fc41686eab7400b796b/raw/b575bd01a58494dfddc1d6429ef0167e709abf9b/hamlet.txt")
h = h.read()
h = h.decode("utf-8")
regex = re.compile('[^a-zA-Z]')
h = regex.sub('', h)
h = h.lower()


In [ ]:
len(h)

130135

This version of Hamlet is somewhat shorter than expected :p

In [ ]:
print(c(h))
print(c(h)/avg(len(h)))

671538513081564733020901249
68.75728915700326


That is about 70 times more WATERLOO than what we expected. Maybe it is because of letter frequency of English [0]?
For sure most texts would not contain as many Zs as Es...

Now let's see what happens when we experiment with slightly more realistic distribution of alphabets

[0] https://www3.nd.edu/~busiforc/handouts/cryptography/letterfrequencies.html

In [ ]:
import numpy as np

def ta2(n,t):
  l = []
  a = list('eariotnslcudpmhgbfywkvxzjq')
  P = [0.1116,0.085,0.0758,0.0754,0.0716,0.0695,0.0665,0.0574,0.054900000000000004,0.0454,0.0363,0.0338,0.0317,0.0301,0.03,0.024700000000000003,0.0207,0.0181,0.0178,0.0129,0.011000000000000001,0.0101,0.0029,0.0027,0.002,0.002]
  P = np.array(P)
  P = P/P.sum()
  for _ in range(n):
    l.append(c(np.array([np.random.choice(a,p=P) for _ in range(t)])))
  return mean(l)

print(c(h))
print(f'{ta2(20,132680):.25f}')

671538513081564733020901249
433013491352115525009276928.0000000000000000000000000


We are getting there but there is one more bias in human language : surely letters do not appear independently from each other.

So we shuffle the text of Hamlet :

In [ ]:
L = []
for _ in range(20):
    l = list(h)
    random.shuffle(l)
    hr = ''.join(l)
    L.append(c(hr))
ahr = mean(L)
print(f'{ahr:.25f}')

782135088086677818427047936.0000000000000000000000000


And we try to take account for actual letter frequencies of Hamelet

In [ ]:
from collections import Counter
counts=Counter(h)
print(counts.items())


dict_items([('t', 11863), ('h', 8731), ('e', 14960), ('r', 7777), ('a', 9950), ('g', 2420), ('d', 5025), ('y', 3204), ('o', 11218), ('f', 2698), ('m', 4253), ('l', 5847), ('p', 2016), ('i', 8511), ('n', 8297), ('c', 2606), ('k', 1272), ('b', 1830), ('w', 3132), ('s', 8379), ('u', 4343), ('v', 1222), ('z', 72), ('q', 220), ('x', 179), ('j', 110)])


In [ ]:
a2 = []
p2 = []
for p in counts.items():
    a2 = a2 + [p[0]]
    p2 = p2 + [p[1]/len(h)]
print(a2)
print(p2)

['t', 'h', 'e', 'r', 'a', 'g', 'd', 'y', 'o', 'f', 'm', 'l', 'p', 'i', 'n', 'c', 'k', 'b', 'w', 's', 'u', 'v', 'z', 'q', 'x', 'j']
[0.0911591808506551, 0.06709186613900948, 0.11495754408883083, 0.059761017405002496, 0.07645906174357398, 0.01859607330848734, 0.03861374726245822, 0.024620586314212163, 0.08620278941099627, 0.020732316440619358, 0.032681446190494484, 0.04493026472509317, 0.015491604871863834, 0.0654013140200561, 0.06375686786798325, 0.0200253582817843, 0.009774464978675991, 0.014062319898566872, 0.0240673147116456, 0.06438698274868405, 0.033373035693702695, 0.009390248588004765, 0.0005532716025665655, 0.0016905521189533946, 0.0013754946786029892, 0.0008452760594766973]


And generate random strings based on that

In [ ]:
def ta3(n,t):
  l = []
  P = p2
  P = np.array(P)
  P = P/P.sum()
  for _ in range(n):
    l.append(c(np.array([np.random.choice(a2,p=P) for _ in range(t)])))
  return mean(l)

print(f'{ahr:.25f}')
print(f'{ta3(50,len(h)):.25f}')

782135088086677818427047936.0000000000000000000000000
780735900078762857437593600.0000000000000000000000000


Aaaannnnd that is close enough for me